<div  >
<img src="https://raw.githubusercontent.com/threeML/astromodels/master/docs/media/transp_logo.png" alt="drawing" width="300" align="right"/>
 


<div  >
<img src="https://raw.githubusercontent.com/threeML/threeML/master/logo/logo_sq.png" alt="drawing" width="300" align="right"/>



# Advanced Topics

Here we will cover a few more examples beyond just x-ray analsysis to get an idea of what is possible in 3ML.


 



## Joint Fits

We have learned to do x-ray analysis with 3ML, but one of the main purposes of 3ML is to allow joint fits with different types of data. To demostrate this, we will have a look at some simulated optical and x-ray data from a GRB afterglow observered by Chandra and the 7-Band GROND instrument.

<img src="https://room.eu.com/images/contents/GRBurst.jpg" alt="drawing" width="400" align="left"/>
<img src="https://cdn.eso.org/images/screen/eso0730a.jpg" alt="drawing" width="400" align="right"/>







In [ ]:
from threeML import *


# get xspec models
#from astromodels.xspec import *
update_logging_level("INFO")
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

%matplotlib notebook
from jupyterthemes import jtplot
jtplot.style(context='notebook', fscale=1, ticks=True, grid=False)

from pathlib import Path

data_path = Path("joint_data")

### Photometric Data

Photometric data is handled in 3ML via the `PhotometricLike` plugin. We use the `speclite` package to interface to a variety of filters that come preloaded in 3ML. This is updated all the time, but it is already pretty extensive. If you want to use the filters, you need to load them manually:


In [ ]:
import speclite.filters as spec_filters

from threeML.utils.photometry import PhotometericObservation
# The filter library takes a while to load so you must import it explicitly.
from threeML.utils.photometry import get_photometric_filter_library

threeML_filter_library = get_photometric_filter_library()

Let's get the filters for GROND and have a look

In [ ]:
grond_filters = threeML_filter_library.LaSilla.GROND

In [ ]:
spec_filters.plot_filters(grond_filters)

The shapes of the filters are important. Many times, astronomers try to convert magnitude to counts via multiplying by the average filter response. This causes the errors to become inaccurate. 3ML works directly in magnitude space via *synthetic photometry*, i.e., forward folding the photon model into the filter response to provide more accurate fits. 


There are several ways to load data into the plugin.

1) via keywords where data are passed as tuples of magnitudes and uncertainties with the keyword being the band name:

In [ ]:
grond_observation = PhotometericObservation.from_kwargs(g=(21.5,0.02),
                                                        r=(22.,0.03),
                                                        i=(21.8, 0.01),
                                                        z=(21.2, 0.01),
                                                        J=(19.6, 0.1),
                                                        H=(18.6, 0.01),
                                                        K=(18.0, 0.1))

grond_observation

2) via an HDF5 file type in 3ML that stores the data:

In [ ]:
grond_observation = PhotometericObservation.from_hdf5("joint_data/grond_data.h5")

We can then load these observation into the plugin, or we could use the helper functions to do this directly

In [ ]:
grond = PhotometryLike.from_file("grond",grond_filters,file_name="joint_data/grond_data.h5",)
grond.plot();

We can see what data are in the plugin:

In [ ]:
grond.band_J

and we can turn the bands on and off for the fit:

In [ ]:
grond.band_J.off = True

In [ ]:
grond.band_J

In [ ]:
grond.band_J.on = True

In [ ]:
grond.band_J

### X-ray data

Now we can load our x-ray data as we always do:


In [ ]:
chandra = OGIPLike("chandra",observation="joint_data/obs_demo.pha", spectrum_number=1)

chandra.set_active_measurements("1-10")

chandra.rebin_on_background(1)

chandra.view_count_spectrum();


### Performing a joint fit

To do a joint fit, everything is the same as before except that now our DataList is a little bigger


In [ ]:
data = DataList(grond, chandra)

This time we will use maximum likelihood for the fun of it. GRB afterglows are typically characterized as broken power laws to approximate their synchrotron radiation. Let's build a model for that:

In [ ]:
bpl = Broken_powerlaw(K=1, piv=1)

# set some reasonable bounds on the break energy
bpl.xb.bounds = (0, None)

ps = PointSource("grb", 0,0, spectral_shape=bpl)


model = Model(ps)

In [ ]:
jl = JointLikelihood(model, data)

# let's use the ROOT minimizer
jl.set_minimizer("ROOT")

In [ ]:
_ = jl.fit()

Because these data live in different spaces, they have different x-axes. Thus, we have to plot the data fits with different functions:

In [ ]:
display_spectrum_model_counts(jl, min_rate=10);

In [ ]:
display_photometry_model_magnitudes(jl);

Of course, we can always look at the model space:

In [ ]:
plot_spectra(jl.results, ene_min =25000 * u.Angstrom, ene_max = 1.239842* u.Angstrom, energy_unit=u.Angstrom  , flux_unit="erg/s/cm2/keV");

But maybe it is a power law?

In [ ]:
pl = Powerlaw(K=1, piv=1)

ps_pl = PointSource("grb_pl", 0,0, spectral_shape=pl)
model_pl = Model(ps_pl)

In [ ]:
jl_pl = JointLikelihood(model_pl, data)

In [ ]:
jl_pl.set_minimizer("ROOT")

In [ ]:
_ = jl_pl.fit(quiet=False)

In [ ]:
plot_spectra(jl.results, jl_pl.results ,ene_min =25000 * u.Angstrom, ene_max = 1.239842* u.Angstrom, energy_unit=u.Angstrom  , flux_unit="erg/s/cm2/keV");

#### Comparing models with the likelihood ratio test. 

We have a rather complex likelihood and we really do not want to trust Wilks' theorem to give us the true "significance" of the more advanced model. Thus, we need to calibrate our LRT via monte carlo. Once of the best papers on statistical calibration is [this one](https://arxiv.org/abs/1803.08393) by Michael Betancourt. 3ML provides some syntax magic to do this for you.

**NOTE:** This operation can be very slow on the binder environment. I recommend only running it locally. The `with parallel_computation` command can start small cluster on your computer and perform the operations in parallel.



In [ ]:
lrt = LikelihoodRatioTest(jl_pl,jl)

In [ ]:
with parallel_computation(start_cluster=True):

    lrt.by_mc(100, continue_on_failure=True)

We can see the results of the LRT by plotting them

In [ ]:
lrt.plot_TS_distribution(bins=100, ec="k", fc="white", lw=1.2)
_ = plt.legend()

## Complex Models


### Generating some synthetic data

![alt text](http://aasnova.org/wp-content/uploads/2016/03/fig16.jpg)

Let's say we have two galactic x-ray sources, some accreting compact binaries perhaps? We observe them at two different times. These sources (imaginary) sources emit a blackbody which is theorized to always be at the same temperature, but perhaps at different flux levels.


Lets simulate one of these sources:




In [ ]:
from threeML.utils.OGIP.response import OGIPResponse
from threeML.io.package_data import get_path_of_data_file

In [ ]:
np.random.seed(1234)

# we will use a demo response
response_1 = OGIPResponse(get_path_of_data_file("datasets/ogip_powerlaw.rsp"))


source_function_1 = Blackbody(K=5e-8, kT=500.0)
background_function_1 = Powerlaw(K=1, index=-1.5, piv=1.0e3)


spectrum_generator_1 = DispersionSpectrumLike.from_function(
    "s1",
    source_function=source_function_1,
    background_function=background_function_1,
    response=response_1,
)

fig = spectrum_generator_1.view_count_spectrum()

Now let's simulate the other source, but this one has an extra feature! There is a power law component in addition to the blackbody. 

In [ ]:
response_2 = OGIPResponse(get_path_of_data_file("datasets/ogip_powerlaw.rsp"))


source_function_2 = Blackbody(K=1e-7, kT=500.0) + Powerlaw_flux(
    F=2e2, index=-1.5, a=10, b=500
)
background_function_2 = Powerlaw(K=1, index=-1.5, piv=1.0e3)


spectrum_generator_2 = DispersionSpectrumLike.from_function(
    "s2",
    source_function=source_function_2,
    background_function=background_function_2,
    response=response_2,
)

fig = spectrum_generator_2.view_count_spectrum()

### Make the model

Now let's make the model we will use to fit the data. First, let's make the spectral function for source_1 and set priors on the parameters.



In [ ]:
spectrum_1 = Blackbody()

spectrum_1.K.prior = Log_normal(mu=np.log(1e-7), sigma=1)
spectrum_1.kT.prior = Log_normal(mu=np.log(300), sigma=2)

ps1 = PointSource("src1", ra=1, dec=20, spectral_shape=spectrum_1)

We will do the same for the other source but also include the power law component 

In [ ]:
spectrum_2 = Blackbody() + Powerlaw_flux(
    a=10, b=500
)  # a,b are the bounds for the flux for this model

spectrum_2.K_1.prior = Log_normal(mu=np.log(1e-6), sigma=1)
spectrum_2.kT_1.prior = Log_normal(mu=np.log(300), sigma=2)

spectrum_2.F_2.prior = Log_normal(mu=np.log(1e2), sigma=1)
spectrum_2.F_2.bounds = (None, None)

spectrum_2.index_2.prior = Gaussian(mu=-1.0, sigma=1)
spectrum_2.index_2.bounds = (None, None)

ps2 = PointSource("src2", ra=2, dec=-10, spectral_shape=spectrum_2)

In [ ]:
model = Model(ps1, ps2)


### Linking parameters

We hypothesized that both sources should have the a same blackbody temperature. We can impose this by linking the temperatures. 



In [ ]:
model.link(
    model.src1.spectrum.main.Blackbody.kT, model.src2.spectrum.main.composite.kT_1
)

we could also link the parameters with an arbitrary function rather than directly. Check out the [astromodels documentation](https://astromodels.readthedocs.io/en/latest/Model_tutorial.html#linking-parameters) for more details.

In [ ]:
model


### Assigning sources to plugins

Now, if we simply passed out model to the BayesianAnalysis or JointLikelihood objects, it would sum the point source spectra together and apply both sources to all data. 

This is not what we want. Many plugins have the ability to be assigned directly to a source. Let's do that here:

In [ ]:
spectrum_generator_1.assign_to_source("src1")

spectrum_generator_2.assign_to_source("src2")

Now we simply make our our data list

In [ ]:
data = DataList(spectrum_generator_1, spectrum_generator_2)


### Fitting the data

Now we fit the data as we normally would. We use Bayesian analysis here.

In [ ]:
ba = BayesianAnalysis(model, data)
ba.set_sampler("multinest")
ba.sampler.setup(n_live_points=500)
#ba.sampler.setup(frac_remain=0.5)
_ = ba.sample()

Let's examine the fits.

In [ ]:
fig = display_spectrum_model_counts(ba)
ax = fig.get_axes()[0]
ax.set_ylim(1e-6)

Lets grab the result. Remember, we can save the results to disk, so all of the following operations can be run at a later time without having to redo all the above steps!

In [ ]:
result = ba.results
fig = result.corner_plot()


## Computing fluxes

Now we will compute fluxes. We can compute them in many different units, over any energy range also specified in any units. 

The flux is computed by integrating the function over energy. By default, a fast trapezoid method is used. If you need more accuracy, you can change the method in the configuration.

In [ ]:
silence_warnings()

# we could also use 'quad' here, but it is slower

threeML_config.point_source.integrate_flux_method = "trapz"

result.get_flux(ene_min=1 * u.keV, ene_max=1 * u.MeV, flux_unit="erg/cm2/s")

We see that a pandas dataframe is returned with all the information. We could change the confidence region for the uncertainties if we desire. However, we could also sum the source fluxes! 3ML will take care of propagating the uncertainties (for any of these operations). 

In [ ]:


result.get_flux(
    ene_min=1 * u.keV,
    ene_max=1 * u.MeV,
    confidence_level=0.95,
    sum_sources=True,
    flux_unit="erg/cm2/s",
)

We can get the fluxes of individual components:

In [ ]:
result.get_flux(
    ene_min=10 * u.keV, ene_max=0.5 * u.MeV, use_components=True, flux_unit="1/(cm2 s)"
)

As well as choose which component to compute

In [ ]:
result.get_flux(ene_min=10*u.keV, 
                ene_max = 0.5*u.MeV,
                use_components=True,
                components_to_use =["Blackbody"],
                
                flux_unit="erg2/(cm2 s)")

Finally, the returned flux object is a pandas table and can be manipulated as such:

In [ ]:
flux = result.get_flux(ene_min=1 * u.keV, ene_max=1 * u.MeV, flux_unit="J2/m2/yr")


In [ ]:
flux["flux"]

In [ ]:
flux["flux"]["src1: total"]
